In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import gc

In [2]:
datetime.datetime.now().date().weekday()

1

In [3]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
            
last_sturday = (datetime.datetime.now()-datetime.timedelta(days=(datetime.datetime.now().weekday()+2))).date()
last_sturday

datetime.date(2019, 7, 13)

In [4]:
last_day_of_2018Q4=datetime.date(2019,2,2)

year_of_quarter=(last_sturday-last_day_of_2018Q4).days/(52*7)
year_of_quarter=str(int(2019+np.floor(year_of_quarter)))
print("Year",year_of_quarter)


quarter_of_quarter=(last_sturday-last_day_of_2018Q4).days/7
quarter_of_quarter=np.floor(quarter_of_quarter/13)%4
quarter_of_quarter=str(int(1+quarter_of_quarter))
print("Quarter",quarter_of_quarter)

str_current_quarter=year_of_quarter+"_Q"+quarter_of_quarter

print(str_current_quarter)

current_week=int((last_sturday-last_day_of_2018Q4).days/7%13)
print("current_week",current_week)

Year 2019
Quarter 2
2019_Q2
current_week 10


In [5]:
current_quarter_beginning=last_day_of_2018Q4+datetime.timedelta(days=(int(year_of_quarter)-2019)*52*7+\
                                                                (int(quarter_of_quarter)-1)*13*7+1)
year_start=datetime.date(last_sturday.year,1,1)  

print("current_quarter_beginning",current_quarter_beginning)
print("year_start",year_start)

current_quarter_beginning 2019-05-05
year_start 2019-01-01


In [6]:
all_2019_new_sign_ups=list(recursive_file_gen("/home/jian/BigLots/"))
all_2019_new_sign_ups=[x for x in all_2019_new_sign_ups if "aster" in x]
all_2019_new_sign_ups=[x for x in all_2019_new_sign_ups if "MediaStorm_"+str(last_sturday.year) in x]

df_all_2019_new_sign_ups=pd.DataFrame({"file_path":all_2019_new_sign_ups})
df_all_2019_new_sign_ups['week_end_dt']=df_all_2019_new_sign_ups['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
df_all_2019_new_sign_ups=df_all_2019_new_sign_ups.sort_values("week_end_dt",ascending=True)
df_all_2019_new_sign_ups=df_all_2019_new_sign_ups[df_all_2019_new_sign_ups['week_end_dt']>="2019-06-08"]
df_all_2019_new_sign_ups.shape

(6, 2)

In [7]:
all_2019_daily_sales=list(recursive_file_gen("/home/jian/BigLots/"))
all_2019_daily_sales=[x for x in all_2019_daily_sales if "aily" in x]
all_2019_daily_sales=[x for x in all_2019_daily_sales if "MediaStorm_"+str(last_sturday.year) in x]

df_all_2019_daily_sales=pd.DataFrame({"file_path":all_2019_daily_sales})
df_all_2019_daily_sales['week_end_dt']=df_all_2019_daily_sales['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
df_all_2019_daily_sales=df_all_2019_daily_sales.sort_values("week_end_dt",ascending=True)

df_all_2019_daily_sales.shape

(28, 2)

In [8]:
df_cum_new_rewards_year_0=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                       compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
df_cum_new_rewards_year_0=df_cum_new_rewards_year_0[df_cum_new_rewards_year_0['sign_up_date']>="2019-01-01"]
df_cum_new_rewards_year_1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                       compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
df_cum_new_rewards_year=df_cum_new_rewards_year_1.append(df_cum_new_rewards_year_0)
df_cum_new_rewards_year=df_cum_new_rewards_year.sort_values("sign_up_date",ascending=False).drop_duplicates("customer_id_hashed")

############
df_cum_new_rewards_quarter=df_cum_new_rewards_year[df_cum_new_rewards_year['sign_up_date']>=str(current_quarter_beginning)]


del df_cum_new_rewards_year_0
del df_cum_new_rewards_year_1
gc.collect()

28

In [9]:
for file_new_sign_ups in df_all_2019_new_sign_ups['file_path'].tolist():
    df=pd.read_csv(file_new_sign_ups,dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    df=df[df['sign_up_date'].apply(lambda x: x[:4]==str(last_sturday.year))]
    df_cum_new_rewards_year=df_cum_new_rewards_year.append(df)
    
    df=df[df['sign_up_date']>=str(current_quarter_beginning)]
    df_cum_new_rewards_quarter=df_cum_new_rewards_quarter.append(df)
    
df_new_rewards_week=df.copy()


In [10]:
# Rewards Only

df_cum_daily_sales_year=pd.DataFrame()
df_cum_daily_sales_quarter=pd.DataFrame()

i_counter=0
for file_daily_sales in df_all_2019_daily_sales['file_path'].tolist():
    df=pd.read_csv(file_daily_sales,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt']).drop_duplicates()
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df[df['transaction_dt'].apply(lambda x: x[:4]==str(last_sturday.year))]
    df_cum_daily_sales_year=df_cum_daily_sales_year.append(df)
    del df_cum_daily_sales_year['transaction_dt']
    
    df=df[df['transaction_dt']>=str(current_quarter_beginning)]
    df_cum_daily_sales_quarter=df_cum_daily_sales_quarter.append(df)
    del df_cum_daily_sales_quarter['transaction_dt']
    i_counter+=1
    
    if i_counter%5==1:
        print(i_counter,datetime.datetime.now())
df_cum_daily_sales_year=df_cum_daily_sales_year.drop_duplicates()
df_cum_daily_sales_quarter=df_cum_daily_sales_quarter.drop_duplicates()

df_daily_sales_week=df[['customer_id_hashed']].drop_duplicates()


1 2019-07-16 16:53:34.551105
6 2019-07-16 16:59:52.811221
11 2019-07-16 17:02:18.580095
16 2019-07-16 17:05:01.155368
21 2019-07-16 17:07:16.853152
26 2019-07-16 17:09:35.945485


In [11]:
df_cum_daily_sales_quarter.head(2)

,customer_id_hashed
0,3ea01138bc00e68a5115a81574bcfbc5d30e407ea9f954...
1,aff71f09a856d7a5aba235ec3b4d9d43cd3815275c1eda...


In [12]:
df_cum_daily_sales_year['Shoppers']="Purchased"
df_cum_daily_sales_quarter['Shoppers']="Purchased"
df_daily_sales_week['Shoppers']="Purchased"

df_cum_new_rewards_year=pd.merge(df_cum_new_rewards_year,df_cum_daily_sales_year,on="customer_id_hashed",how="left")
df_cum_new_rewards_quarter=pd.merge(df_cum_new_rewards_quarter,df_cum_daily_sales_quarter,on="customer_id_hashed",how="left")
df_new_rewards_week=pd.merge(df_new_rewards_week,df_daily_sales_week,on="customer_id_hashed",how="left")


In [13]:
week_end_dt=str(last_sturday)

rewards_shoppers_in_week=df_daily_sales_week.shape[0]
rewards_shoppers_cum_quarter=df_cum_daily_sales_quarter.shape[0]
rewards_shoppers_cum_year=df_cum_daily_sales_year.shape[0]

new_sign_ups_cum_year=df_cum_new_rewards_year.shape[0]
new_signed_shoppers_cum_year=df_cum_new_rewards_year[pd.notnull(df_cum_new_rewards_year['Shoppers'])].shape[0]

new_sign_ups_cum_quarter=df_cum_new_rewards_quarter.shape[0]
new_signed_shoppers_cum_quarter=df_cum_new_rewards_quarter[pd.notnull(df_cum_new_rewards_quarter['Shoppers'])].shape[0]

new_signed_week=df_new_rewards_week.shape[0]
new_signed_shoppers_week=df_new_rewards_week[pd.notnull(df_new_rewards_week['Shoppers'])].shape[0]

date_begin_year=str(year_start)
date_begin_quarter=str(current_quarter_beginning)
Current_Quarter=str_current_quarter


In [14]:
df_output_this_week=pd.DataFrame({"week_end_dt":week_end_dt,
                                 "rewards_shoppers_in_week":rewards_shoppers_in_week,
                                 "rewards_shoppers_cum_quarter":rewards_shoppers_cum_quarter,
                                 "rewards_shoppers_cum_year":rewards_shoppers_cum_year,
                                 "new_sign_ups_cum_year":new_sign_ups_cum_year,
                                 "new_signed_shoppers_cum_year":new_signed_shoppers_cum_year,
                                 "new_sign_ups_cum_quarter":new_sign_ups_cum_quarter,
                                 "new_signed_shoppers_cum_quarter":new_signed_shoppers_cum_quarter,
                                 "new_signed_week":new_signed_week,
                                 "new_signed_shoppers_week":new_signed_shoppers_week,
                                 "date_begin_year":date_begin_year,
                                 "date_begin_quarter":date_begin_quarter,
                                 "Current_Quarter":Current_Quarter},index=[0])
df_output_this_week=df_output_this_week[['week_end_dt','rewards_shoppers_in_week','rewards_shoppers_cum_quarter','rewards_shoppers_cum_year',
                                        'new_sign_ups_cum_year','new_signed_shoppers_cum_year','new_sign_ups_cum_quarter','new_signed_shoppers_cum_quarter',
                                        'new_signed_week','new_signed_shoppers_week','date_begin_year','date_begin_quarter','Current_Quarter']]

In [15]:
previous_week_end_dt=(datetime.datetime.strptime(week_end_dt,"%Y-%m-%d")-datetime.timedelta(days=7)).date()
previous_week_end_dt=str(previous_week_end_dt)
previous_week_end_dt

'2019-07-06'

In [16]:
df_output_previous_week=pd.read_csv("/home/simeng/outputs_"+previous_week_end_dt+"/"+"New_rewards_df_this_week_"+previous_week_end_dt+".csv")

cum_this_week_output=df_output_previous_week.append(df_output_this_week)               

cum_this_week_output

,week_end_dt,rewards_shoppers_in_week,rewards_shoppers_cum_quarter,rewards_shoppers_cum_year,new_sign_ups_cum_year,new_signed_shoppers_cum_year,new_sign_ups_cum_quarter,new_signed_shoppers_cum_quarter,new_signed_week,new_signed_shoppers_week,date_begin_year,date_begin_quarter,Current_Quarter
0,2019-05-18,1129115,2094109,10518161,1601373,1581323,147779,145050,73460,71931,2019-01-01,2019-05-05,2019_Q2
1,2019-05-25,1251796,3001086,10851684,1686955,1665886,233361,229366,85582,84031,2019-01-01,2019-05-05,2019_Q2
2,2019-06-01,1251328,3801919,11179929,1773934,1751937,320340,315194,86979,85525,2019-01-01,2019-05-05,2019_Q2
3,2019-06-08,1214947,4493724,11479074,2787080,1906675,577624,403728,116429,81908,2019-01-01,2019-05-05,2019_Q2
4,2019-06-15,1243896,5144205,11780498,2908215,1994935,698759,489846,121135,83569,2019-01-01,2019-05-05,2019_Q2
5,2019-06-22,1155770,5692909,12047132,3023349,2077104,813893,570222,115134,77854,2019-01-01,2019-05-05,2019_Q2
6,2019-06-29,1232019,6244242,12328085,3144907,2164501,935451,655834,121558,82934,2019-01-01,2019-05-05,2019_Q2
7,2019-07-06,1309524,6805095,12629325,3276782,2260045,1067326,749436,131875,90436,2019-01-01,2019-05-05,2019_Q2
0,2019-07-13,1436237,7412175,12980523,3455064,2401091,1245608,888198,178282,134495,2019-01-01,2019-05-05,2019_Q2


In [17]:
cum_this_week_output.to_csv("/home/simeng/outputs_"+week_end_dt+"/"+"New_rewards_df_this_week_"+week_end_dt+".csv",index=False)
cum_this_week_output.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/JT_2_rolling_new_sign_ups_count/output_table_1/"+"New_rewards_df_this_week_"+week_end_dt+".csv",index=False)


In [18]:
cum_this_week_output

,week_end_dt,rewards_shoppers_in_week,rewards_shoppers_cum_quarter,rewards_shoppers_cum_year,new_sign_ups_cum_year,new_signed_shoppers_cum_year,new_sign_ups_cum_quarter,new_signed_shoppers_cum_quarter,new_signed_week,new_signed_shoppers_week,date_begin_year,date_begin_quarter,Current_Quarter
0,2019-05-18,1129115,2094109,10518161,1601373,1581323,147779,145050,73460,71931,2019-01-01,2019-05-05,2019_Q2
1,2019-05-25,1251796,3001086,10851684,1686955,1665886,233361,229366,85582,84031,2019-01-01,2019-05-05,2019_Q2
2,2019-06-01,1251328,3801919,11179929,1773934,1751937,320340,315194,86979,85525,2019-01-01,2019-05-05,2019_Q2
3,2019-06-08,1214947,4493724,11479074,2787080,1906675,577624,403728,116429,81908,2019-01-01,2019-05-05,2019_Q2
4,2019-06-15,1243896,5144205,11780498,2908215,1994935,698759,489846,121135,83569,2019-01-01,2019-05-05,2019_Q2
5,2019-06-22,1155770,5692909,12047132,3023349,2077104,813893,570222,115134,77854,2019-01-01,2019-05-05,2019_Q2
6,2019-06-29,1232019,6244242,12328085,3144907,2164501,935451,655834,121558,82934,2019-01-01,2019-05-05,2019_Q2
7,2019-07-06,1309524,6805095,12629325,3276782,2260045,1067326,749436,131875,90436,2019-01-01,2019-05-05,2019_Q2
0,2019-07-13,1436237,7412175,12980523,3455064,2401091,1245608,888198,178282,134495,2019-01-01,2019-05-05,2019_Q2


# Email

In [19]:
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
import datetime
import os

In [20]:
sender="jubapluscc@gmail.com"
receivers='jthomas@mediastorm.biz, jian@jubaplus.com'
subject="Big Lots New Sign Ups Report Qumulative of Year/Quarter/Week"

In [21]:
file = "/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/JT_2_rolling_new_sign_ups_count/output_table_1/"+"New_rewards_df_this_week_"+week_end_dt+".csv"
text_message_str="./email_message_new_rewards_weekly.txt"

msg = MIMEMultipart()
msg['From'] = sender
msg['To'] = receivers
msg['Date'] = formatdate(localtime=True)
msg['Subject'] = subject
with open(text_message_str,'r') as f:
    text_mesaage = f.read()
msg.attach(MIMEText(text_mesaage))

with open(file,'rb') as attachment:
    att = MIMEApplication(
        attachment.read(),name=os.path.basename(file)
    )
    att['Content-Disposition'] = 'attachment; filename="%s"' %os.path.basename(file)
    msg.attach(att)



smtp = smtplib.SMTP('smtp.gmail.com',587)
smtp.ehlo()
smtp.starttls()
smtp.login(sender,"jubaplus2015")
smtp.send_message(msg)

smtp.close()